# Capstone Project 2: Scientific Computing - Numerical Simulations

## Coupled Oscillator System and Population Dynamics

This capstone project demonstrates scientific computing techniques using NumPy and SciPy. We'll explore two classic physics/biology simulations:

1. **Damped Harmonic Oscillator**: A fundamental physics system
2. **Coupled Pendulum System**: Multiple interacting oscillators
3. **Predator-Prey Dynamics**: The Lotka-Volterra equations

### Learning Objectives
- Solve ordinary differential equations (ODEs) numerically
- Use SciPy's integration functions
- Visualize dynamical systems with animations and phase plots
- Explore parameter space and sensitivity analysis
- Apply scientific computing best practices

### Prerequisites
- Understanding of calculus (derivatives)
- Basic physics concepts (forces, motion)
- NumPy array operations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Circle
from scipy.integrate import odeint, solve_ivp
from scipy.signal import find_peaks
from scipy.optimize import minimize_scalar
from typing import Tuple, Callable, List
from dataclasses import dataclass
import warnings

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
np.set_printoptions(precision=4)

print("Scientific Computing Environment Ready!")
print(f"NumPy version: {np.__version__}")
print("\nTopics covered in this notebook:")
print("  1. Damped Harmonic Oscillator")
print("  2. Coupled Pendulum System")
print("  3. Predator-Prey Population Dynamics")

---
## Part 1: Damped Harmonic Oscillator

### The Physics

The damped harmonic oscillator is described by the equation:

$$m\ddot{x} + c\dot{x} + kx = F(t)$$

Where:
- $m$ is mass
- $c$ is the damping coefficient
- $k$ is the spring constant
- $F(t)$ is an external driving force

We can rewrite this as:

$$\ddot{x} + 2\gamma\dot{x} + \omega_0^2 x = f(t)$$

Where $\gamma = c/(2m)$ is the damping ratio and $\omega_0 = \sqrt{k/m}$ is the natural frequency.

### Converting to First-Order System

For numerical integration, we convert to a system of first-order ODEs:

$$\dot{x} = v$$
$$\dot{v} = -2\gamma v - \omega_0^2 x + f(t)$$

In [ ]:
@dataclass
class OscillatorParams:
    """
    Parameters for a damped harmonic oscillator.
    
    Attributes
    ----------
    omega_0 : float
        Natural angular frequency (rad/s)
    gamma : float
        Damping ratio
    driving_amplitude : float
        Amplitude of external driving force
    driving_freq : float
        Frequency of external driving force (rad/s)
    """
    omega_0: float = 1.0
    gamma: float = 0.1
    driving_amplitude: float = 0.0
    driving_freq: float = 1.0
    
    @property
    def damping_type(self) -> str:
        """Classify the damping regime."""
        if self.gamma < self.omega_0:
            return "underdamped"
        elif self.gamma > self.omega_0:
            return "overdamped"
        else:
            return "critically damped"
    
    @property
    def damped_frequency(self) -> float:
        """Calculate the damped oscillation frequency."""
        if self.gamma < self.omega_0:
            return np.sqrt(self.omega_0**2 - self.gamma**2)
        return 0.0


def harmonic_oscillator_ode(
    state: np.ndarray, 
    t: float, 
    params: OscillatorParams
) -> np.ndarray:
    """
    Define the ODEs for a damped, driven harmonic oscillator.
    
    Parameters
    ----------
    state : np.ndarray
        Current state [position, velocity]
    t : float
        Current time
    params : OscillatorParams
        System parameters
    
    Returns
    -------
    np.ndarray
        Derivatives [dx/dt, dv/dt]
    """
    x, v = state
    
    # External driving force
    f = params.driving_amplitude * np.cos(params.driving_freq * t)
    
    # Equations of motion
    dx_dt = v
    dv_dt = -2 * params.gamma * v - params.omega_0**2 * x + f
    
    return np.array([dx_dt, dv_dt])


def simulate_oscillator(
    params: OscillatorParams,
    initial_state: Tuple[float, float] = (1.0, 0.0),
    t_span: Tuple[float, float] = (0, 50),
    n_points: int = 1000
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Simulate the harmonic oscillator.
    
    Parameters
    ----------
    params : OscillatorParams
        System parameters
    initial_state : tuple
        Initial (position, velocity)
    t_span : tuple
        Time interval (start, end)
    n_points : int
        Number of time points
    
    Returns
    -------
    tuple
        (time array, position array, velocity array)
    """
    t = np.linspace(t_span[0], t_span[1], n_points)
    
    # Solve using odeint
    solution = odeint(
        harmonic_oscillator_ode,
        initial_state,
        t,
        args=(params,)
    )
    
    return t, solution[:, 0], solution[:, 1]


# Create default parameters and simulate
params = OscillatorParams(omega_0=1.0, gamma=0.1)
print(f"System: {params.damping_type}")
print(f"Natural frequency: {params.omega_0:.2f} rad/s")
print(f"Damped frequency: {params.damped_frequency:.4f} rad/s")
print(f"Damping ratio: {params.gamma}")

In [ ]:
# Compare different damping regimes
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Different damping scenarios
damping_cases = [
    OscillatorParams(omega_0=1.0, gamma=0.1),   # Underdamped
    OscillatorParams(omega_0=1.0, gamma=1.0),   # Critically damped
    OscillatorParams(omega_0=1.0, gamma=2.0),   # Overdamped
]

colors = ['steelblue', 'coral', 'forestgreen']

# Time domain comparison
ax1 = axes[0, 0]
for params, color in zip(damping_cases, colors):
    t, x, v = simulate_oscillator(params, t_span=(0, 30))
    ax1.plot(t, x, color=color, label=f'{params.damping_type} (gamma={params.gamma})', linewidth=2)

ax1.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Position')
ax1.set_title('Damping Regime Comparison')
ax1.legend()
ax1.set_xlim(0, 30)

# Phase space (position vs velocity)
ax2 = axes[0, 1]
for params, color in zip(damping_cases, colors):
    t, x, v = simulate_oscillator(params, t_span=(0, 30))
    ax2.plot(x, v, color=color, label=f'{params.damping_type}', linewidth=1.5, alpha=0.8)
    # Mark starting point
    ax2.scatter([x[0]], [v[0]], color=color, s=100, zorder=5, marker='o')
    # Mark ending point
    ax2.scatter([x[-1]], [v[-1]], color=color, s=100, zorder=5, marker='s')

ax2.set_xlabel('Position')
ax2.set_ylabel('Velocity')
ax2.set_title('Phase Space Trajectories')
ax2.legend()
ax2.axhline(y=0, color='gray', linestyle='--', alpha=0.3)
ax2.axvline(x=0, color='gray', linestyle='--', alpha=0.3)

# Energy decay
ax3 = axes[1, 0]
for params, color in zip(damping_cases, colors):
    t, x, v = simulate_oscillator(params, t_span=(0, 30))
    # Total energy: E = 0.5*v^2 + 0.5*omega_0^2*x^2
    energy = 0.5 * v**2 + 0.5 * params.omega_0**2 * x**2
    ax3.semilogy(t, energy, color=color, label=f'{params.damping_type}', linewidth=2)

ax3.set_xlabel('Time (s)')
ax3.set_ylabel('Energy (log scale)')
ax3.set_title('Energy Decay')
ax3.legend()

# Driven oscillator - resonance demonstration
ax4 = axes[1, 1]
driving_freqs = np.linspace(0.1, 3.0, 50)
amplitudes = []

base_params = OscillatorParams(omega_0=1.0, gamma=0.1, driving_amplitude=0.1)

for omega_d in driving_freqs:
    params = OscillatorParams(
        omega_0=base_params.omega_0,
        gamma=base_params.gamma,
        driving_amplitude=base_params.driving_amplitude,
        driving_freq=omega_d
    )
    t, x, v = simulate_oscillator(params, t_span=(0, 200), n_points=2000)
    # Get steady-state amplitude (last 20% of simulation)
    steady_state = x[int(len(x)*0.8):]
    amplitudes.append(np.max(np.abs(steady_state)))

ax4.plot(driving_freqs, amplitudes, 'b-', linewidth=2)
ax4.axvline(x=1.0, color='red', linestyle='--', label='Natural frequency')
ax4.set_xlabel('Driving Frequency (rad/s)')
ax4.set_ylabel('Steady-State Amplitude')
ax4.set_title('Resonance Curve')
ax4.legend()

plt.tight_layout()
plt.show()

print(f"\nResonance peak at omega_d = {driving_freqs[np.argmax(amplitudes)]:.2f} rad/s")
print(f"Maximum amplitude: {max(amplitudes):.2f}")

---
## Part 2: Coupled Pendulum System

### The Physics

Two pendulums connected by a spring exhibit fascinating coupled behavior. The equations of motion for small angles are:

$$\ddot{\theta}_1 = -\frac{g}{L}\theta_1 - \frac{k}{m}(\theta_1 - \theta_2)$$
$$\ddot{\theta}_2 = -\frac{g}{L}\theta_2 - \frac{k}{m}(\theta_2 - \theta_1)$$

This system exhibits two normal modes:
1. **In-phase mode**: Both pendulums swing together
2. **Out-of-phase mode**: Pendulums swing in opposite directions

We'll observe the energy transfer phenomenon known as "beating."

In [ ]:
@dataclass
class CoupledPendulumParams:
    """
    Parameters for coupled pendulum system.
    
    Attributes
    ----------
    length : float
        Pendulum length (m)
    mass : float
        Bob mass (kg)
    spring_k : float
        Coupling spring constant (N/m)
    g : float
        Gravitational acceleration (m/s^2)
    """
    length: float = 1.0
    mass: float = 1.0
    spring_k: float = 0.5
    g: float = 9.81
    
    @property
    def omega_pendulum(self) -> float:
        """Natural frequency of uncoupled pendulum."""
        return np.sqrt(self.g / self.length)
    
    @property
    def omega_spring(self) -> float:
        """Coupling frequency from spring."""
        return np.sqrt(self.spring_k / self.mass)
    
    @property
    def omega_in_phase(self) -> float:
        """Frequency of in-phase normal mode."""
        return self.omega_pendulum
    
    @property
    def omega_out_phase(self) -> float:
        """Frequency of out-of-phase normal mode."""
        return np.sqrt(self.omega_pendulum**2 + 2 * self.omega_spring**2)
    
    @property
    def beat_frequency(self) -> float:
        """Beat frequency (energy transfer rate)."""
        return abs(self.omega_out_phase - self.omega_in_phase) / 2


def coupled_pendulum_ode(
    state: np.ndarray,
    t: float,
    params: CoupledPendulumParams
) -> np.ndarray:
    """
    ODEs for coupled pendulum system (small angle approximation).
    
    Parameters
    ----------
    state : np.ndarray
        State vector [theta1, omega1, theta2, omega2]
    t : float
        Time
    params : CoupledPendulumParams
        System parameters
    
    Returns
    -------
    np.ndarray
        Derivatives
    """
    theta1, omega1, theta2, omega2 = state
    
    # Coupling term
    omega_p2 = params.omega_pendulum**2
    omega_s2 = params.omega_spring**2
    
    d_theta1 = omega1
    d_omega1 = -omega_p2 * theta1 - omega_s2 * (theta1 - theta2)
    d_theta2 = omega2
    d_omega2 = -omega_p2 * theta2 - omega_s2 * (theta2 - theta1)
    
    return np.array([d_theta1, d_omega1, d_theta2, d_omega2])


def simulate_coupled_pendulum(
    params: CoupledPendulumParams,
    initial_state: Tuple[float, float, float, float] = (0.3, 0, 0, 0),
    t_span: Tuple[float, float] = (0, 60),
    n_points: int = 2000
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Simulate the coupled pendulum system.
    
    Parameters
    ----------
    params : CoupledPendulumParams
        System parameters
    initial_state : tuple
        Initial (theta1, omega1, theta2, omega2)
    t_span : tuple
        Time interval
    n_points : int
        Number of time points
    
    Returns
    -------
    tuple
        (time array, solution array)
    """
    t = np.linspace(t_span[0], t_span[1], n_points)
    
    solution = odeint(
        coupled_pendulum_ode,
        initial_state,
        t,
        args=(params,)
    )
    
    return t, solution


# Create system and display parameters
cp_params = CoupledPendulumParams(length=1.0, mass=1.0, spring_k=0.5)
print("Coupled Pendulum System Parameters:")
print(f"  Pendulum length: {cp_params.length} m")
print(f"  Mass: {cp_params.mass} kg")
print(f"  Spring constant: {cp_params.spring_k} N/m")
print(f"\nCharacteristic Frequencies:")
print(f"  Pendulum omega: {cp_params.omega_pendulum:.4f} rad/s")
print(f"  Spring omega: {cp_params.omega_spring:.4f} rad/s")
print(f"  In-phase mode: {cp_params.omega_in_phase:.4f} rad/s")
print(f"  Out-of-phase mode: {cp_params.omega_out_phase:.4f} rad/s")
print(f"  Beat frequency: {cp_params.beat_frequency:.4f} rad/s")
print(f"  Beat period: {2*np.pi/cp_params.beat_frequency:.2f} s")

In [ ]:
# Simulate and visualize coupled pendulum dynamics
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Case 1: Only first pendulum displaced (demonstrates beating)
t, sol = simulate_coupled_pendulum(cp_params, initial_state=(0.3, 0, 0, 0), t_span=(0, 60))
theta1, omega1, theta2, omega2 = sol.T

# Time evolution
ax1 = axes[0, 0]
ax1.plot(t, theta1, 'b-', label='Pendulum 1', linewidth=1.5)
ax1.plot(t, theta2, 'r-', label='Pendulum 2', linewidth=1.5)
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Angle (rad)')
ax1.set_title('Energy Transfer Between Pendulums (Beating)')
ax1.legend()
ax1.set_xlim(0, 60)

# Energy of each pendulum
ax2 = axes[0, 1]
E1 = 0.5 * cp_params.mass * cp_params.length**2 * omega1**2 + \
     0.5 * cp_params.mass * cp_params.g * cp_params.length * theta1**2
E2 = 0.5 * cp_params.mass * cp_params.length**2 * omega2**2 + \
     0.5 * cp_params.mass * cp_params.g * cp_params.length * theta2**2

ax2.plot(t, E1, 'b-', label='Pendulum 1', linewidth=1.5)
ax2.plot(t, E2, 'r-', label='Pendulum 2', linewidth=1.5)
ax2.plot(t, E1 + E2, 'g--', label='Total', linewidth=1.5, alpha=0.7)
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Energy (J)')
ax2.set_title('Energy Transfer Dynamics')
ax2.legend()

# Normal modes - In-phase
ax3 = axes[1, 0]
t_ip, sol_ip = simulate_coupled_pendulum(cp_params, initial_state=(0.2, 0, 0.2, 0), t_span=(0, 20))
ax3.plot(t_ip, sol_ip[:, 0], 'b-', label='Pendulum 1', linewidth=2)
ax3.plot(t_ip, sol_ip[:, 2], 'r--', label='Pendulum 2', linewidth=2)
ax3.set_xlabel('Time (s)')
ax3.set_ylabel('Angle (rad)')
ax3.set_title('In-Phase Normal Mode')
ax3.legend()

# Normal modes - Out-of-phase
ax4 = axes[1, 1]
t_op, sol_op = simulate_coupled_pendulum(cp_params, initial_state=(0.2, 0, -0.2, 0), t_span=(0, 20))
ax4.plot(t_op, sol_op[:, 0], 'b-', label='Pendulum 1', linewidth=2)
ax4.plot(t_op, sol_op[:, 2], 'r--', label='Pendulum 2', linewidth=2)
ax4.set_xlabel('Time (s)')
ax4.set_ylabel('Angle (rad)')
ax4.set_title('Out-of-Phase Normal Mode')
ax4.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Parameter exploration: Effect of coupling strength
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

spring_constants = [0.1, 0.5, 2.0]
labels = ['Weak coupling (k=0.1)', 'Medium coupling (k=0.5)', 'Strong coupling (k=2.0)']
colors = ['steelblue', 'coral', 'forestgreen']

for ax, k, label, color in zip(axes, spring_constants, labels, colors):
    params = CoupledPendulumParams(spring_k=k)
    t, sol = simulate_coupled_pendulum(params, initial_state=(0.3, 0, 0, 0), t_span=(0, 60))
    
    ax.plot(t, sol[:, 0], color=color, alpha=0.8, label='Pendulum 1', linewidth=1.5)
    ax.plot(t, sol[:, 2], color='gray', alpha=0.6, label='Pendulum 2', linewidth=1.5)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Angle (rad)')
    ax.set_title(f'{label}\nBeat period: {2*np.pi/params.beat_frequency:.1f}s')
    ax.legend()
    ax.set_xlim(0, 60)

plt.tight_layout()
plt.show()

print("\nObservation: Stronger coupling leads to faster energy transfer (shorter beat period)")

---
## Part 3: Predator-Prey Population Dynamics

### The Lotka-Volterra Equations

The classic predator-prey model describes the interaction between two species:

$$\frac{dx}{dt} = \alpha x - \beta xy$$
$$\frac{dy}{dt} = \delta xy - \gamma y$$

Where:
- $x$ = prey population (e.g., rabbits)
- $y$ = predator population (e.g., foxes)
- $\alpha$ = prey birth rate
- $\beta$ = predation rate
- $\delta$ = predator reproduction rate (per prey eaten)
- $\gamma$ = predator death rate

This system exhibits oscillatory behavior where predator and prey populations cycle out of phase.

In [ ]:
@dataclass
class LotkaVolterraParams:
    """
    Parameters for Lotka-Volterra predator-prey model.
    
    Attributes
    ----------
    alpha : float
        Prey birth rate
    beta : float
        Predation rate coefficient
    delta : float
        Predator reproduction efficiency
    gamma : float
        Predator death rate
    """
    alpha: float = 1.0    # Prey birth rate
    beta: float = 0.1     # Predation rate
    delta: float = 0.075  # Predator reproduction rate
    gamma: float = 1.5    # Predator death rate
    
    @property
    def equilibrium_prey(self) -> float:
        """Equilibrium prey population."""
        return self.gamma / self.delta
    
    @property
    def equilibrium_predator(self) -> float:
        """Equilibrium predator population."""
        return self.alpha / self.beta


def lotka_volterra_ode(
    state: np.ndarray,
    t: float,
    params: LotkaVolterraParams
) -> np.ndarray:
    """
    Lotka-Volterra predator-prey ODEs.
    
    Parameters
    ----------
    state : np.ndarray
        Current populations [prey, predator]
    t : float
        Time (not used explicitly, but required by solver)
    params : LotkaVolterraParams
        Model parameters
    
    Returns
    -------
    np.ndarray
        Population derivatives [d(prey)/dt, d(predator)/dt]
    """
    x, y = state  # x = prey, y = predator
    
    dx_dt = params.alpha * x - params.beta * x * y
    dy_dt = params.delta * x * y - params.gamma * y
    
    return np.array([dx_dt, dy_dt])


def simulate_population(
    params: LotkaVolterraParams,
    initial_pop: Tuple[float, float] = (40, 9),
    t_span: Tuple[float, float] = (0, 50),
    n_points: int = 2000
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Simulate population dynamics.
    
    Parameters
    ----------
    params : LotkaVolterraParams
        Model parameters
    initial_pop : tuple
        Initial (prey, predator) populations
    t_span : tuple
        Time interval
    n_points : int
        Number of time points
    
    Returns
    -------
    tuple
        (time, prey population, predator population)
    """
    t = np.linspace(t_span[0], t_span[1], n_points)
    
    solution = odeint(
        lotka_volterra_ode,
        initial_pop,
        t,
        args=(params,)
    )
    
    return t, solution[:, 0], solution[:, 1]


# Create parameters
lv_params = LotkaVolterraParams(alpha=1.0, beta=0.1, delta=0.075, gamma=1.5)
print("Lotka-Volterra Model Parameters:")
print(f"  Prey birth rate (alpha): {lv_params.alpha}")
print(f"  Predation rate (beta): {lv_params.beta}")
print(f"  Predator efficiency (delta): {lv_params.delta}")
print(f"  Predator death rate (gamma): {lv_params.gamma}")
print(f"\nEquilibrium populations:")
print(f"  Prey: {lv_params.equilibrium_prey:.1f}")
print(f"  Predator: {lv_params.equilibrium_predator:.1f}")

In [ ]:
# Simulate and visualize population dynamics
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Main simulation
t, prey, predator = simulate_population(lv_params, initial_pop=(40, 9), t_span=(0, 50))

# Time series
ax1 = axes[0, 0]
ax1.plot(t, prey, 'b-', label='Prey (Rabbits)', linewidth=2)
ax1.plot(t, predator, 'r-', label='Predator (Foxes)', linewidth=2)
ax1.axhline(y=lv_params.equilibrium_prey, color='blue', linestyle='--', alpha=0.5, label='Prey equilibrium')
ax1.axhline(y=lv_params.equilibrium_predator, color='red', linestyle='--', alpha=0.5, label='Predator equilibrium')
ax1.set_xlabel('Time')
ax1.set_ylabel('Population')
ax1.set_title('Population Dynamics Over Time')
ax1.legend()
ax1.set_xlim(0, 50)

# Phase space
ax2 = axes[0, 1]
ax2.plot(prey, predator, 'purple', linewidth=1.5, alpha=0.8)
ax2.scatter([prey[0]], [predator[0]], color='green', s=100, zorder=5, label='Start', marker='o')
ax2.scatter([lv_params.equilibrium_prey], [lv_params.equilibrium_predator], 
            color='red', s=150, zorder=5, label='Equilibrium', marker='*')
ax2.set_xlabel('Prey Population')
ax2.set_ylabel('Predator Population')
ax2.set_title('Phase Space (Predator vs Prey)')
ax2.legend()

# Add direction arrows to phase plot
arrow_indices = np.linspace(0, len(t)-2, 10, dtype=int)
for i in arrow_indices:
    ax2.annotate('', xy=(prey[i+1], predator[i+1]), xytext=(prey[i], predator[i]),
                arrowprops=dict(arrowstyle='->', color='gray', alpha=0.5))

# Multiple initial conditions - phase portrait
ax3 = axes[1, 0]
initial_conditions = [
    (20, 5), (40, 5), (60, 5),
    (20, 15), (40, 15), (60, 15)
]
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(initial_conditions)))

for ic, color in zip(initial_conditions, colors):
    t_ic, prey_ic, pred_ic = simulate_population(lv_params, initial_pop=ic, t_span=(0, 30))
    ax3.plot(prey_ic, pred_ic, color=color, alpha=0.7, linewidth=1.5)
    ax3.scatter([ic[0]], [ic[1]], color=color, s=50, zorder=5)

ax3.scatter([lv_params.equilibrium_prey], [lv_params.equilibrium_predator], 
            color='red', s=150, zorder=10, marker='*', label='Equilibrium')
ax3.set_xlabel('Prey Population')
ax3.set_ylabel('Predator Population')
ax3.set_title('Phase Portrait (Multiple Initial Conditions)')
ax3.legend()

# Population ratio over time
ax4 = axes[1, 1]
ratio = predator / prey
ax4.plot(t, ratio, 'purple', linewidth=2)
ax4.axhline(y=lv_params.equilibrium_predator/lv_params.equilibrium_prey, 
            color='red', linestyle='--', label='Equilibrium ratio')
ax4.set_xlabel('Time')
ax4.set_ylabel('Predator/Prey Ratio')
ax4.set_title('Population Ratio Oscillation')
ax4.legend()
ax4.set_xlim(0, 50)

plt.tight_layout()
plt.show()

In [ ]:
# Parameter sensitivity analysis
def analyze_sensitivity(
    base_params: LotkaVolterraParams,
    param_name: str,
    param_range: np.ndarray
) -> dict:
    """
    Analyze sensitivity of oscillation characteristics to parameter changes.
    
    Parameters
    ----------
    base_params : LotkaVolterraParams
        Base parameter set
    param_name : str
        Name of parameter to vary
    param_range : np.ndarray
        Range of parameter values to test
    
    Returns
    -------
    dict
        Dictionary with period and amplitude data
    """
    periods = []
    max_prey = []
    max_predator = []
    
    for val in param_range:
        # Create modified parameters
        params_dict = {
            'alpha': base_params.alpha,
            'beta': base_params.beta,
            'delta': base_params.delta,
            'gamma': base_params.gamma
        }
        params_dict[param_name] = val
        params = LotkaVolterraParams(**params_dict)
        
        # Simulate
        t, prey, predator = simulate_population(params, initial_pop=(40, 9), t_span=(0, 100), n_points=5000)
        
        # Find peaks to estimate period
        peaks, _ = find_peaks(prey[len(prey)//2:])  # Use second half for steady state
        if len(peaks) >= 2:
            avg_period = np.mean(np.diff(peaks)) * (t[1] - t[0])
            periods.append(avg_period)
        else:
            periods.append(np.nan)
        
        # Max populations (steady state)
        max_prey.append(np.max(prey[len(prey)//2:]))
        max_predator.append(np.max(predator[len(predator)//2:]))
    
    return {
        'param_values': param_range,
        'periods': np.array(periods),
        'max_prey': np.array(max_prey),
        'max_predator': np.array(max_predator)
    }


# Analyze sensitivity to each parameter
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Alpha sensitivity
alpha_range = np.linspace(0.5, 2.0, 20)
alpha_results = analyze_sensitivity(lv_params, 'alpha', alpha_range)

ax1 = axes[0, 0]
ax1.plot(alpha_results['param_values'], alpha_results['periods'], 'o-', color='steelblue')
ax1.set_xlabel('Prey birth rate (alpha)')
ax1.set_ylabel('Oscillation Period')
ax1.set_title('Sensitivity to Prey Birth Rate')

# Beta sensitivity
beta_range = np.linspace(0.05, 0.2, 20)
beta_results = analyze_sensitivity(lv_params, 'beta', beta_range)

ax2 = axes[0, 1]
ax2.plot(beta_results['param_values'], beta_results['max_prey'], 'b-', label='Max Prey')
ax2.plot(beta_results['param_values'], beta_results['max_predator'], 'r-', label='Max Predator')
ax2.set_xlabel('Predation rate (beta)')
ax2.set_ylabel('Maximum Population')
ax2.set_title('Sensitivity to Predation Rate')
ax2.legend()

# Gamma sensitivity (predator death rate)
gamma_range = np.linspace(0.5, 3.0, 20)
gamma_results = analyze_sensitivity(lv_params, 'gamma', gamma_range)

ax3 = axes[1, 0]
ax3.plot(gamma_results['param_values'], gamma_results['periods'], 'o-', color='coral')
ax3.set_xlabel('Predator death rate (gamma)')
ax3.set_ylabel('Oscillation Period')
ax3.set_title('Sensitivity to Predator Death Rate')

# Combined effect visualization
ax4 = axes[1, 1]
# Vary both alpha and gamma
alphas = [0.5, 1.0, 1.5]
for alpha in alphas:
    params = LotkaVolterraParams(alpha=alpha)
    t, prey, predator = simulate_population(params, initial_pop=(40, 9), t_span=(0, 30))
    ax4.plot(prey, predator, label=f'alpha={alpha}', linewidth=1.5)

ax4.set_xlabel('Prey Population')
ax4.set_ylabel('Predator Population')
ax4.set_title('Phase Portraits with Different Birth Rates')
ax4.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Conservation law verification
# The Lotka-Volterra equations have a conserved quantity (constant of motion)

def lotka_volterra_conserved_quantity(
    prey: np.ndarray, 
    predator: np.ndarray, 
    params: LotkaVolterraParams
) -> np.ndarray:
    """
    Calculate the conserved quantity V for Lotka-Volterra system.
    
    V = delta*x - gamma*ln(x) + beta*y - alpha*ln(y)
    
    This quantity remains constant along trajectories.
    
    Parameters
    ----------
    prey : np.ndarray
        Prey population values
    predator : np.ndarray
        Predator population values
    params : LotkaVolterraParams
        Model parameters
    
    Returns
    -------
    np.ndarray
        Conserved quantity values
    """
    V = (params.delta * prey - params.gamma * np.log(prey) + 
         params.beta * predator - params.alpha * np.log(predator))
    return V


# Verify conservation
t, prey, predator = simulate_population(lv_params, initial_pop=(40, 9), t_span=(0, 50))
V = lotka_volterra_conserved_quantity(prey, predator, lv_params)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
ax1.plot(t, V, 'green', linewidth=2)
ax1.set_xlabel('Time')
ax1.set_ylabel('Conserved Quantity V')
ax1.set_title('Conservation Law Verification')
ax1.set_ylim(V.mean() - 0.5, V.mean() + 0.5)

ax2 = axes[1]
# Plot contours of V in phase space
prey_grid = np.linspace(5, 100, 100)
pred_grid = np.linspace(2, 25, 100)
X, Y = np.meshgrid(prey_grid, pred_grid)
V_grid = lotka_volterra_conserved_quantity(X, Y, lv_params)

contour = ax2.contour(X, Y, V_grid, levels=20, cmap='viridis')
ax2.plot(prey, predator, 'r-', linewidth=2, label='Trajectory')
ax2.scatter([lv_params.equilibrium_prey], [lv_params.equilibrium_predator], 
            color='red', s=100, zorder=5, marker='*')
ax2.set_xlabel('Prey Population')
ax2.set_ylabel('Predator Population')
ax2.set_title('Contours of Conserved Quantity')
ax2.legend()
plt.colorbar(contour, ax=ax2, label='V')

plt.tight_layout()
plt.show()

print(f"\nConserved quantity V:")
print(f"  Mean: {V.mean():.6f}")
print(f"  Standard deviation: {V.std():.6f}")
print(f"  Relative variation: {V.std()/V.mean()*100:.4f}%")
print("\nThe small variation is due to numerical integration errors.")

---
## Part 4: Advanced Analysis - Stability and Bifurcation

Let's examine the stability of fixed points and how the system behavior changes with parameters.

In [ ]:
def compute_jacobian_lv(
    x: float, 
    y: float, 
    params: LotkaVolterraParams
) -> np.ndarray:
    """
    Compute the Jacobian matrix of the Lotka-Volterra system.
    
    The Jacobian matrix J at point (x, y) is:
    J = [[alpha - beta*y,  -beta*x],
         [delta*y,         delta*x - gamma]]
    
    Parameters
    ----------
    x : float
        Prey population
    y : float
        Predator population
    params : LotkaVolterraParams
        Model parameters
    
    Returns
    -------
    np.ndarray
        2x2 Jacobian matrix
    """
    J = np.array([
        [params.alpha - params.beta * y, -params.beta * x],
        [params.delta * y, params.delta * x - params.gamma]
    ])
    return J


def analyze_fixed_point(
    x: float, 
    y: float, 
    params: LotkaVolterraParams
) -> dict:
    """
    Analyze the stability of a fixed point.
    
    Parameters
    ----------
    x : float
        Prey population at fixed point
    y : float
        Predator population at fixed point
    params : LotkaVolterraParams
        Model parameters
    
    Returns
    -------
    dict
        Analysis results including eigenvalues and stability type
    """
    J = compute_jacobian_lv(x, y, params)
    eigenvalues = np.linalg.eigvals(J)
    
    # Classify stability
    real_parts = eigenvalues.real
    imag_parts = eigenvalues.imag
    
    if np.all(real_parts < 0):
        if np.any(imag_parts != 0):
            stability = "stable spiral (damped oscillations)"
        else:
            stability = "stable node"
    elif np.all(real_parts > 0):
        if np.any(imag_parts != 0):
            stability = "unstable spiral"
        else:
            stability = "unstable node"
    elif np.all(real_parts == 0) and np.any(imag_parts != 0):
        stability = "center (neutral stability, oscillations)"
    else:
        stability = "saddle point"
    
    return {
        'jacobian': J,
        'eigenvalues': eigenvalues,
        'stability': stability,
        'trace': np.trace(J),
        'determinant': np.linalg.det(J)
    }


# Analyze fixed points
print("="*60)
print("FIXED POINT ANALYSIS")
print("="*60)

# Fixed point 1: Origin (trivial)
print("\n1. Trivial Fixed Point (0, 0):")
result_origin = analyze_fixed_point(0.01, 0.01, lv_params)  # Near origin
print(f"   Eigenvalues: {result_origin['eigenvalues']}")
print(f"   Stability: {result_origin['stability']}")

# Fixed point 2: Coexistence equilibrium
print(f"\n2. Coexistence Fixed Point ({lv_params.equilibrium_prey:.2f}, {lv_params.equilibrium_predator:.2f}):")
result_eq = analyze_fixed_point(lv_params.equilibrium_prey, lv_params.equilibrium_predator, lv_params)
print(f"   Jacobian matrix:")
print(f"   {result_eq['jacobian']}")
print(f"   Eigenvalues: {result_eq['eigenvalues']}")
print(f"   Stability: {result_eq['stability']}")
print(f"   Trace: {result_eq['trace']:.6f}")
print(f"   Determinant: {result_eq['determinant']:.6f}")

In [ ]:
# Vector field visualization
fig, ax = plt.subplots(figsize=(10, 8))

# Create grid for vector field
x = np.linspace(5, 80, 20)
y = np.linspace(2, 20, 20)
X, Y = np.meshgrid(x, y)

# Calculate derivatives at each point
U = lv_params.alpha * X - lv_params.beta * X * Y
V = lv_params.delta * X * Y - lv_params.gamma * Y

# Normalize for better visualization
magnitude = np.sqrt(U**2 + V**2)
U_norm = U / magnitude
V_norm = V / magnitude

# Plot vector field
ax.quiver(X, Y, U_norm, V_norm, magnitude, cmap='coolwarm', alpha=0.8)

# Plot nullclines
# Prey nullcline: dx/dt = 0 => y = alpha/beta
ax.axhline(y=lv_params.alpha/lv_params.beta, color='blue', linestyle='--', 
           label=f'Prey nullcline (y={lv_params.alpha/lv_params.beta:.1f})', linewidth=2)

# Predator nullcline: dy/dt = 0 => x = gamma/delta
ax.axvline(x=lv_params.gamma/lv_params.delta, color='red', linestyle='--', 
           label=f'Predator nullcline (x={lv_params.gamma/lv_params.delta:.1f})', linewidth=2)

# Plot a few trajectories
initial_conditions = [(20, 5), (60, 15), (40, 3)]
for ic in initial_conditions:
    t_traj, prey_traj, pred_traj = simulate_population(lv_params, initial_pop=ic, t_span=(0, 20))
    ax.plot(prey_traj, pred_traj, 'k-', linewidth=1.5, alpha=0.7)
    ax.scatter([ic[0]], [ic[1]], color='green', s=80, zorder=5)

# Mark equilibrium
ax.scatter([lv_params.equilibrium_prey], [lv_params.equilibrium_predator], 
           color='gold', s=200, zorder=10, marker='*', edgecolors='black', linewidths=2)

ax.set_xlabel('Prey Population', fontsize=12)
ax.set_ylabel('Predator Population', fontsize=12)
ax.set_title('Phase Portrait with Vector Field and Nullclines', fontsize=14)
ax.legend(loc='upper right')
ax.set_xlim(5, 80)
ax.set_ylim(2, 20)

plt.tight_layout()
plt.show()

---
## Summary and Conclusions

This capstone project demonstrated scientific computing techniques through three interconnected simulations:

### Skills Applied

1. **Numerical Integration**
   - Converting second-order ODEs to first-order systems
   - Using `scipy.integrate.odeint` for reliable integration
   - Understanding solver parameters and accuracy

2. **Physical Modeling**
   - Damped harmonic oscillator: fundamental mechanics
   - Coupled systems: normal modes and energy transfer
   - Population dynamics: ecological modeling

3. **Visualization Techniques**
   - Time series plots
   - Phase space trajectories
   - Vector fields and nullclines
   - Parameter sensitivity plots

4. **Analysis Methods**
   - Linear stability analysis via Jacobian
   - Eigenvalue classification of fixed points
   - Conservation law verification
   - Parameter sensitivity studies

### Key Insights

1. **Damped Oscillator**: The damping ratio determines qualitative behavior (underdamped, critically damped, overdamped)
2. **Coupled Pendulums**: Energy transfer period depends on coupling strength
3. **Predator-Prey**: Closed orbits in phase space reflect the conservation law

### Further Exploration

- Add nonlinear damping or driving forces
- Implement more complex predator-prey models (carrying capacity, delays)
- Create interactive animations with matplotlib or Plotly
- Explore chaotic systems (double pendulum, Lorenz equations)

In [ ]:
print("="*60)
print("       SCIENTIFIC COMPUTING CAPSTONE COMPLETE!")
print("="*60)
print("\nYou have learned to:")
print("  [x] Formulate physical problems as ODEs")
print("  [x] Solve ODEs numerically with SciPy")
print("  [x] Visualize dynamical systems")
print("  [x] Analyze stability of fixed points")
print("  [x] Perform parameter sensitivity analysis")
print("  [x] Verify conservation laws numerically")